In [13]:
import sys
sys.path.insert(0, r'../')
import syspy

import geopandas as gpd
import pandas as pd
ipath = r'inputs/'

In [14]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 10)

## Nearest neighbor

In [15]:
nodes = gpd.read_file(ipath + r'nodes.geojson')
clusters = gpd.read_file(ipath + r'clusters.geojson')

In [16]:
from syspy.syspy_utils import neighbors

In [17]:
neighbors.nearest(nodes, clusters)

,ix_one,index_nn,ix_many,distance,rank
0,0,37,37,0.000000,0
1,1,36,36,289.079677,0
119,119,36,36,241.070869,0
409,409,36,36,172.366801,0
734,734,36,36,138.051547,0
...,...,...,...,...,...
946,946,155,155,124.699954,0
894,894,162,162,95.214438,0
941,941,162,162,95.214438,0
919,919,56,56,15.142848,0


In [18]:
nodes['cluster_id'] = neighbors.nearest(nodes, clusters).set_index('ix_one')['ix_many']

## Desire lines

In [19]:
zones = gpd.read_file(ipath + 'zones_neighbors.geojson').set_index('zone_id')
ods = pd.read_csv(ipath + r'ods.csv', index_col=0)

In [20]:
neighborhood = neighbors.Neighborhood(
    zones.to_crs(epsg=4236), # only works in lat lon
    ods,
    ['volume']
)

FloatProgress(value=0.0, max=8.0)

neighborhood_dataframe: 100%|██████████| 370/370 [00:03<00:00, 107.57it/s]


In [21]:
od_geoms = neighborhood.export(
    'volume',
    affected=False,
)

No projection defined --> considered as EPSG:4326
can only map affected volumes with arg : od_geometry=False


In [22]:
od_geoms.columns

Index(['origin', 'destination', 'volume', 'link', 'link_list',
       'volume_transit', 'latitude_origin', 'longitude_origin',
       'latitude_destination', 'longitude_destination', 'geometry', 'distance',
       'label', 'color', 'width'],
      dtype='object')

In [23]:
from shapely.geometry import LineString

od_geoms['geometry'] = od_geoms.apply(
    lambda x: LineString(
        [
            (x['longitude_origin'], x['latitude_origin']),
            (x['longitude_destination'], x['latitude_destination'])
        ]
    ), 
    1
)

od_geoms = gpd.GeoDataFrame(od_geoms)[
    ['origin', 'destination', 'volume',
    'geometry', 'label', 'color', 'width']
]
od_geoms.to_file(
    ipath + r'desire_lines/non_affected_ods.geojson', driver='GeoJSON'
)

In [24]:
desire_lines = neighborhood.export(
    'volume',
    affected=True,
)
desire_lines = gpd.GeoDataFrame(desire_lines)[
    ['origin', 'destination', 'volume', 'volume_transit',
    'geometry', 'label', 'color', 'width']
]
desire_lines.to_file(
    ipath + r'desire_lines/affected_ods.geojson', driver='GeoJSON'
)

No projection defined --> considered as EPSG:4326


# classified desire lines

In [25]:
from syspy.spatial import spatial
cluster_geoms, cluster_ids = spatial.zone_clusters(
    zones, n_clusters=4
)
zones['cluster'] = cluster_ids

In [26]:
neighbors.zones_classified_neighborhood(
    zones, ods, ipath + r'desire_lines/macro/', 'cluster', volume_col='volume'
)


2


FloatProgress(value=0.0, max=8.0)

neighborhood_dataframe: 100%|██████████| 370/370 [00:03<00:00, 120.59it/s]


3


FloatProgress(value=0.0, max=8.0)

...

FloatProgress(value=0.0, max=8.0)

neighborhood_dataframe: 100%|██████████| 370/370 [00:03<00:00, 110.74it/s]


0


FloatProgress(value=0.0, max=8.0)

neighborhood_dataframe: 100%|██████████| 370/370 [00:02<00:00, 131.10it/s]
4it [00:00,  4.24it/s]
